# import package

In [1]:
from pathlib import Path

import pandas as pd

from attribute_extraction.utils.att_lov_extract import (
    get_classification_report,
    get_performance_metrics_table,
)

# Setting up variables

In [2]:
attribute_code_col = "attribute_code"
attribute_label_col = "attribute_label"
attribute_lov_col = "lov_code"
language_col = "lang"

pred_label_col = "predicted_labels"

predictions_path = "../outputs_predict_workflow/predictions.csv"


# Loading Predictions data

In [3]:

predictions = pd.read_csv(predictions_path).reset_index(drop=True)

In [6]:
predictions

,adeo_product_id,id,title,description,lang,attribute_code,attribute_label,attribute_type,is_multivalued,is_lov,...,step_model_code,mirakl_model_code,data_source,description_clean,context,predicted_labels,predicted_scores,is_top1,is_top3,is_top5
0,84375704,b'\xf2\x94\x92\xf7\x1fJ\xe7\x95&\x8f~s!5\x7fA/...,"Cojines para palets, Respaldo 120x50cm y Asien...","El precio no incluye muebles. Dimensiones, asi...",ES,2419,Estilo,CHAR,0,1,...,202345,202345|COUSSIN_D_EXTERIEUR|COUSSIN_ET_MATELAS_...,BOMP,"El precio no incluye muebles. Dimensiones, asi...","Cojines para palets, Respaldo 120x50cm y Asien...",70116,0.999016,1,1,1
1,86488166,b'\x04\x1a_\xe9wCp\xd2\x9c\xd7m\xabF\x17\x86\x...,Cadre photo Éternel brut 30x40 cm,<p> Protégez et mettez en avant toutes vos pho...,FR,2419,Style,CHAR,0,1,...,202309,202309|CADRE_MONTE_SANS_PASSE-PARTOUT|CADRE_PH...,BOMP,Protégez et mettez en avant toutes vos photos ...,Cadre photo Éternel brut 30x40 cm Protégez et ...,64985,0.678465,1,1,1
2,86161296,b'^6\x8f\xe5\xd8\xde\xc2\x19\xfa\xbb)C\xb6\x01...,"Taie d'oreiller imprimée en percale de coton, ...",Changer d'univers et faire voyager l’esprit et...,FR,2419,Style,CHAR,0,1,...,202592,202592|1038|R12-004-005,BOMP,Changer d'univers et faire voyager l’esprit et...,"Taie d'oreiller imprimée en percale de coton, ...",211348,0.849754,0,1,1
3,84524098,b'?{)pK\x11\xe0`:\x14}H/\xe0\xd4I\x9b\x11d4e\x...,Accessoire KARRI 24 cm - 1 lumière,Accessoire KARRI bleu-24cm- 1 lumière vers le...,FR,2419,Style,CHAR,0,1,...,202375,202375|ABATS-JOUR|ABATS-JOUR|R13-001-024,BOMP,Accessoire KARRI bleu-24cm- 1 lumière vers le...,Accessoire KARRI 24 cm - 1 lumière Accessoir...,52870,0.998305,1,1,1
4,84635560,b'\xec\xc7p*\xd8\x1f\xee\x18\xadq\xbf\xcb\xb6\...,Papier peint adhésif - Mechanical Symmetry III...,Nos papiers peints adhésifs sont une propositi...,FR,2419,Style,CHAR,0,1,...,202265,202265|PAPIER_PEINT|PAPIER_PEINT_FRISE_ET_FIBR...,BOMP,Nos papiers peints adhésifs sont une propositi...,Papier peint adhésif - Mechanical Symmetry III...,44028,0.202138,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10961,84201416,b'\xaf\xcf\x12\xbb}}9!\xee\xb5\xdc\rV\xe4B\x16...,Tableau sur verre expression hivernale - 200 x...,<div style='text-align: left;'><p> <strong>Le ...,FR,99999,product_model,CHAR,0,1,...,202301,202301|VERRE_IMPRIME_SANS_ENCADREMENT|AFFICHE_...,BOMP,<div style='text-align: left;'> Le tableau réa...,Tableau sur verre expression hivernale - 200 x...,202301,0.999134,1,1,1
10962,84201416,b'(I\x0c\xfc\xd0\t\x18\xbf\xda\xbc\x8e:VE\xad\...,Tableau sur verre expression hivernale - 200 x...,<div style='text-align: left;'><p> <strong>Le ...,FR,99999,product_model,CHAR,0,1,...,202301,202301|VERRE_IMPRIME_SANS_ENCADREMENT|AFFICHE_...,BOMP,<div style='text-align: left;'> Le tableau réa...,Tableau sur verre expression hivernale - 200 x...,202301,0.999134,1,1,1
10963,83353214,b'Q\xc20\xb4!?\x92Tz\xc5#\xa3\xe3\xf4\xa0\xc1e...,Tableau sur verre pissenlits moelleux - 120 x ...,<div style='text-align: left;'><p> <strong>Le ...,FR,99999,product_model,CHAR,0,1,...,202301,202301|VERRE_IMPRIME_SANS_ENCADREMENT|AFFICHE_...,BOMP,<div style='text-align: left;'> Le tableau réa...,Tableau sur verre pissenlits moelleux - 120 x ...,202301,0.997873,1,1,1
10964,82615344,b'\x001\xd6\xd6\x9f\xeb\xa9\x80(/\x07m\x15\xf1...,Ventilador Torre UFESA TW1500 45W,Si buscas electrodomésticos para tu hogar a lo...,ES,99999,product_model,CHAR,0,1,...,200302,200302|VENTILATEUR_ET_BRASSEUR_D_AIR|CLIMATISE...,BOMP,Si buscas electrodomésticos para tu hogar a lo...,Ventilador Torre UFESA TW1500 45W Si buscas el...,200302,0.991904,1,1,1


In [11]:
attribute_codes = predictions[attribute_code_col].unique()
attribute_codes.sort()

df_print_all = pd.DataFrame(
    columns=[
        attribute_code_col,
        attribute_label_col,
        "Dataframe",
        "Accuracy (%)",
        "Precision (Weighted-avg)(%)",
        "Precision (Macro-avg) (%)",
        "Recall (Weighted-avg) (%)",
        "Recall (Macro - avg) (%)",
        "Number of samples",
    ]
)

for attribute_code in attribute_codes:

    attribute_label = (
        predictions[attribute_label_col]
        .loc[
            (predictions[attribute_code_col] == attribute_code)
            & (predictions[language_col] == "FR")
        ]
        .values[0]
    )

    attribute_data = predictions[predictions[attribute_code_col] == attribute_code]

    df_metric_overall, df_metric_detail = get_classification_report(
        attribute_data[attribute_lov_col], attribute_data[pred_label_col]
    )

    d_eval = {
        "Overall": df_metric_overall,
    }

    for language in ["FR", "ES", "IT"]:

        df_eval_lang = attribute_data.loc[attribute_data[language_col] == language]

        df_metric_overall_lang, df_metric_detail_lang = get_classification_report(
            df_eval_lang[attribute_lov_col], df_eval_lang[pred_label_col]
        )

        d_eval[language] = df_metric_overall_lang

    df_print = get_performance_metrics_table(d_eval)

    df_print["attribute_code"] = str(attribute_code).zfill(5)
    df_print["attribute_label"] = attribute_label

    df_print_all = pd.concat([df_print_all, df_print])



In [12]:
df_print_all

,attribute_code,attribute_label,Dataframe,Accuracy (%),Precision (Weighted-avg)(%),Precision (Macro-avg) (%),Recall (Weighted-avg) (%),Recall (Macro - avg) (%),Number of samples
0,562,Forme,Overall,83.98,84.95,62.28,83.98,69.73,1286
1,562,Forme,FR,84.05,84.61,63.15,84.05,69.99,815
2,562,Forme,ES,84.26,87.03,57.33,84.26,57.09,216
3,562,Forme,IT,83.53,86.31,65.01,83.53,70.89,255
0,1746,Couleur,Overall,56.40,66.65,32.78,56.40,44.34,828
1,1746,Couleur,FR,60.69,70.05,41.85,60.69,50.38,547
2,1746,Couleur,ES,55.73,65.00,29.44,55.73,40.98,131
3,1746,Couleur,IT,41.33,51.50,27.65,41.33,37.79,150
0,2419,Style,Overall,69.89,75.18,47.95,69.89,56.64,1601
1,2419,Style,FR,66.88,73.85,47.06,66.88,54.82,1075
